In [201]:
import pandas as pd
import numpy as np
train_data=pd.read_csv('yds_train2018.csv')
test_data=pd.read_csv('yds_test2018.csv')
submission_format=pd.read_csv('sample_submission.csv')
expances=pd.read_csv('promotional_expense.csv')
holiday=pd.read_excel('holidays.xlsx')

In [202]:
expances.tail()

,Year,Month,Country,Product_Type,Expense_Price
430,2016,6,England,5,426.198
431,2016,6,Finland,4,1289.024
432,2016,7,England,4,157.756
433,2016,7,England,5,378.903
434,2016,7,Finland,4,900.083


In [203]:
expances.columns=[u'Year', u'Month', u'Country', u'Product_ID', u'Expense_Price']

In [204]:
expances['Expense_Price']=expances['Expense_Price']/4

In [205]:
import time
from time import gmtime, strftime
def get_week((x)):
    return(datetime.date((x)).isocalendar()[1])

In [206]:
from datetime import datetime
def strip_date(date_input):
    return(datetime.strptime(date_input, '%Y, %m, %d'))
holiday['Date']=holiday['Date'].apply(lambda x:strip_date(x))

In [207]:
holiday['Week']=holiday['Date'].dt.week

In [208]:
holiday['h_count']=1
holiday=holiday[['Country','Week','h_count']]
holiday=holiday.groupby(['Country','Week']).sum().reset_index()

In [209]:
train_data.head()

,S_No,Year,Month,Week,Merchant_ID,Product_ID,Country,Sales
0,1,2013,1,1,ar00001,1,Argentina,157500.0
1,2,2013,1,1,ar00003,1,Argentina,39375.0
2,3,2013,1,1,ar00004,1,Argentina,15750.0
3,4,2013,1,1,ar00007,1,Argentina,47250.0
4,5,2013,1,1,ar00008,1,Argentina,283500.0


In [210]:
train_data=train_data.drop(['Merchant_ID','S_No'],axis=1)

In [211]:
holiday.head()

,Country,Week,h_count
0,Argentina,1,4
1,Argentina,6,2
2,Argentina,7,4
3,Argentina,8,4
4,Argentina,9,2


In [212]:
train_data=train_data.groupby(['Country','Product_ID','Year','Month','Week']).sum().reset_index()

In [213]:
curr_to_dollar=pd.DataFrame({'Argentina':[0.036],'Belgium':[1.17],'Columbia':[0.00035],'Denmark':[0.16],'England':[1.31],'Finland':[1.17]})
curr_to_dollar=curr_to_dollar.T
curr_to_dollar['Country']=curr_to_dollar.index

In [214]:
curr_to_dollar.head()

,0,Country
Argentina,0.03600,Argentina
Belgium,1.17000,Belgium
Columbia,0.00035,Columbia
Denmark,0.16000,Denmark
England,1.31000,England


In [215]:
train_data=pd.merge(train_data,curr_to_dollar,on='Country',how='left')
train_data['Sales']=train_data['Sales']*train_data[0]
train_data=train_data.drop([0],axis=1)

In [216]:
train_data.head()

,Country,Product_ID,Year,Month,Week,Sales
0,Argentina,1,2013,1,1,156095.1
1,Argentina,1,2013,1,2,351143.1
2,Argentina,1,2013,1,3,371158.2
3,Argentina,1,2013,1,4,269438.4
4,Argentina,1,2013,1,5,88622.1


In [217]:
holiday.head()

,Country,Week,h_count
0,Argentina,1,4
1,Argentina,6,2
2,Argentina,7,4
3,Argentina,8,4
4,Argentina,9,2


In [218]:
train_data=pd.merge(train_data,holiday,on=[u'Country', u'Week'],how='left')

In [219]:
train_data['h_count'].fillna(0,inplace=True)

In [220]:
train_data.columns

Index([u'Country', u'Product_ID', u'Year', u'Month', u'Week', u'Sales',
       u'h_count'],
      dtype='object')

In [221]:
train_data.head()

,Country,Product_ID,Year,Month,Week,Sales,h_count
0,Argentina,1,2013,1,1,156095.1,4.0
1,Argentina,1,2013,1,2,351143.1,0.0
2,Argentina,1,2013,1,3,371158.2,0.0
3,Argentina,1,2013,1,4,269438.4,0.0
4,Argentina,1,2013,1,5,88622.1,0.0


In [222]:
train_data=pd.merge(train_data,expances,on=['Year','Country','Month','Product_ID'],how="left")

In [232]:
train_data['Expense_Price']=train_data.groupby(['Country','Product_ID']).transform(lambda x: x.fillna(x.mean()))

In [233]:
train_data[train_data['Expense_Price'].isnull()]

,Country,Product_ID,Year,Month,Week,Sales,h_count,Expense_Price


In [234]:
features=train_data[[u'Country', u'Product_ID', u'Year', u'Month', u'Week',u'h_count','Expense_Price']]

In [235]:
# from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# train_features, test_features, train_labels, test_labels = train_test_split(features, lebels, test_size = 0.25, random_state = 42)

In [236]:
features=pd.get_dummies(features)

In [237]:
test_features=pd.get_dummies(test_features)

In [238]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse

In [239]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.02
params["min_child_weight"] = 8
params["subsample"] = 0.9
params["colsample_bytree"] = 0.8
params["silent"] = 1
params["max_depth"] = 8
params["seed"] = 1
plst = list(params.items())
num_rounds = 500

In [240]:
xgtrain = xgb.DMatrix(features, lebels)
#xgtest = xgb.DMatrix(te)
model = xgb.train(plst, xgtrain, num_rounds)
#pred_test_y = model.predict(xgtest)

In [241]:
def rmse(act_y, pred_y):
    return np.sqrt(mse(act_y, pred_y))

# test_ data preparation 

In [242]:
test_data=pd.read_csv('yds_test2018.csv')

In [243]:
test_data.columns

Index([u'S_No', u'Year', u'Month', u'Product_ID', u'Country', u'Sales'], dtype='object')

In [244]:
test_data=pd.read_csv('yds_test2018.csv')
test_data1=test_data[[ u'Year', u'Month', u'Product_ID', u'Country', u'Sales']]
test_data2=test_data[[ u'Year', u'Month', u'Product_ID', u'Country', u'Sales']]
test_data3=test_data[[ u'Year', u'Month', u'Product_ID', u'Country', u'Sales']]
test_data4=test_data[[u'Year', u'Month', u'Product_ID', u'Country', u'Sales']]

In [245]:
test_data1['Week']=1


In [246]:
test_data.shape

(105, 6)

In [247]:
test_data1['Week']=1
test_data2['Week']=2

test_data3['Week']=3

test_data4['Week']=4

In [250]:
test_data1=pd.merge(test_data1,holiday,on=[u'Country', u'Week'],how='left')
test_data2=pd.merge(test_data2,holiday,on=[u'Country', u'Week'],how='left')
test_data3=pd.merge(test_data3,holiday,on=[u'Country', u'Week'],how='left')
test_data4=pd.merge(test_data4,holiday,on=[u'Country', u'Week'],how='left')


In [251]:
test_data1.fillna(0,inplace=True)
test_data2.fillna(0,inplace=True)
test_data3.fillna(0,inplace=True)
test_data4.fillna(0,inplace=True)

In [252]:
set(train_data['Product_ID'])

{1, 2, 3, 4, 5}

In [253]:
test_data2.head()

,Year,Month,Product_ID,Country,Sales,Week,h_count
0,2016,4,1,Argentina,0.0,2,0.0
1,2016,5,1,Argentina,0.0,2,0.0
2,2016,6,1,Argentina,0.0,2,0.0
3,2016,7,1,Argentina,0.0,2,0.0
4,2016,8,1,Argentina,0.0,2,0.0


In [254]:
test_data1.columns

Index([u'Year', u'Month', u'Product_ID', u'Country', u'Sales', u'Week',
       u'h_count'],
      dtype='object')

In [255]:
data_list=[test_data1,test_data2,test_data3,test_data4]

In [256]:
test_data1.columns

Index([u'Year', u'Month', u'Product_ID', u'Country', u'Sales', u'Week',
       u'h_count'],
      dtype='object')

In [269]:
test_data['Sales']=0

In [258]:
features.columns

Index([       u'Product_ID',              u'Year',             u'Month',
                    u'Week',           u'h_count',     u'Expense_Price',
       u'Country_Argentina',   u'Country_Belgium',  u'Country_Columbia',
         u'Country_Denmark',   u'Country_England',   u'Country_Finland'],
      dtype='object')

In [259]:
test_data.head()

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,0
1,79074,2016,5,1,Argentina,0
2,79075,2016,6,1,Argentina,0
3,79076,2016,7,1,Argentina,0
4,79077,2016,8,1,Argentina,0


In [263]:
test_features['Country']=test_data1['Country']
test_features=pd.merge(test_features,expances,on=['Year','Country','Month','Product_ID'],how="left")

# feature engineering 

In [270]:
for da in data_list:
    test_features=pd.get_dummies(da)
    test_features['Country']=test_data1['Country']
    test_features=pd.merge(test_features,expances,on=['Year','Country','Month','Product_ID'],how="left")
    test_features['Expense_Price']=test_features.groupby(['Country','Product_ID']).transform(lambda x: x.fillna(x.mean()))
    test_features.drop('Sales',axis=1,inplace=True)
    c_list=[       u'Product_ID',              u'Year',             u'Month',
                    u'Week',           u'h_count',     u'Expense_Price',
       u'Country_Argentina',   u'Country_Belgium',  u'Country_Columbia',
         u'Country_Denmark',   u'Country_England',   u'Country_Finland']
    test_features=test_features[c_list]
    xgtest = xgb.DMatrix(test_features)
    test_features['Sales'] = model.predict(xgtest)
    test_features['Country']=test_data1['Country']
    test_features=pd.merge(test_features,curr_to_dollar,on='Country',how='left')
    test_features['Sales']=test_features['Sales']/test_features[0]
    test_data['Sales']=test_data['Sales']+test_features['Sales']


In [266]:
test_data.to_csv('xgb_submit_more_better.csv',index=False)

In [271]:
test_data.head()

,S_No,Year,Month,Product_ID,Country,Sales
0,79073,2016,4,1,Argentina,1.021052e+07
1,79074,2016,5,1,Argentina,4.560415e+06
2,79075,2016,6,1,Argentina,5.457175e+06
3,79076,2016,7,1,Argentina,3.034447e+06
4,79077,2016,8,1,Argentina,-4.266965e+06
